<a href="https://colab.research.google.com/github/AnaPao1998/Thesis/blob/main/Copy_of_sentiment_analysis_using_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning Roberta for Sentiment Analysis





In [1]:
!pip install transformers==3.0.2

In [2]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

KeyboardInterrupt: ignored

In [ ]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
depression_df = pd.read_csv(io.BytesIO(uploaded['results_tt_depression.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
depression_df.head()

In [ ]:
depression_df.drop(["Unnamed: 0"] ,axis=1, inplace=True)

In [ ]:
depression_df.head()

In [ ]:
depression_df[['pre_processed','depresion']]

In [ ]:
# full_X = depression_df[['pre_processed','depresion']]

# train_size = 0.85
# train_data = full_X.sample(frac=train_size, random_state=200)
# test_data = full_X.drop(train_data.index).reset_index(drop=True)
# train_data = train_data.reset_index(drop=True)


# print(f"No. of training examples: {train_data.shape[0]}")
# print(f"No. of testing examples: {test_data.shape[0]}")

# No. of training examples: 120
# No. of testing examples: 30

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
#from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

full_X = depression_df[['pre_processed', 'depresion']]
X=depression_df[['pre_processed']]
y=depression_df[['depresion']]

# create dataset
#X, y = make_classification(n_samples=100, weights=[0.94], flip_y=0, random_state=1)
#print(Counter(y))

# split into train test sets
train_data, test_data, y_train, y_test = train_test_split(full_X, y, test_size=0.20, random_state=1, stratify=y)
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

print((train_data.shape))
print((test_data.shape))

In [ ]:
train_data[train_data['depresion']==True]

In [ ]:
test_data

In [ ]:
depression_df.shape

<a id='section03'></a>
### Preparing the Dataset and Dataloader

I will start with defining few key variables that will be used later during the training/fine tuning stage.
Followed by creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. I will also define the Dataloader that will feed  the data in batches to the neural network for suitable training and processing. 
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *SentimentData* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the Roberta model for training. 
- I am using the Roberta tokenizer to tokenize the data in the `TITLE` column of the dataframe. 
- The tokenizer uses the `encode_plus` method to perform tokenization and generate the necessary outputs, namely: `ids`, `attention_mask`
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/roberta.html#robertatokenizer)
- `target` is the encoded category on the news headline. 
- The *SentimentData* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.pre_processed
        self.targets = self.data.depresion
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# train_size = 0.8
# train_data=new_df.sample(frac=train_size,random_state=200)
# test_data=new_df.drop(train_data.index).reset_index(drop=True)
# train_data = train_data.reset_index(drop=True)


print(f"FULL Dataset: {full_X.shape}")
print(f"TRAIN Dataset: {train_data.shape}")
print(f"TEST Dataset: {test_data.shape}")

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`. 
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - The data will be fed to the Roberta Language model as defined in the dataset. 
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output. 
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = torch.nn.Softmax()(output)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process. 

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size. 
- Subsequent output from the model and the actual category are compared to calculate the loss. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    
    for i,data in tqdm(enumerate(training_loader, 0)):
        # print(i)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        # big_val, big_idx = torch.max(outputs.data, dim=1)
        pred_tmp =  torch.argmax(outputs.data, dim=1) #torch.max(outputs.data, dim=1)

        n_correct += calcuate_accuracy(pred_tmp, targets)

        

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if i%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            #print(f"Training Loss per 5000 steps: {loss_step}")
            #print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data. 

This unseen data is the 20% of `train.tsv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model. 

As you can see the model is predicting the correct category of a given sample to a 69.47% accuracy which can further be improved by training more.

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testing_loader, 0)):
            # print(i)
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            # big_val, big_idx = torch.max(outputs.data, dim=1)
            pred_tmp =  torch.argmax(outputs.data, dim=1) #torch.max(outputs.data, dim=1)
            # pred_total.append(pred_tmp)
            n_correct += calcuate_accuracy(pred_tmp, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if i%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

In [ ]:
def my_valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    with torch.no_grad():
        pred_total=[]
        targets_total = []
        for i, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            # print(f"the targts is {targets}")
            # print(f"the outputs.data is { outputs.data}")
            # print(f"the outputs.max is { torch.argmax(outputs.data, dim=1)}")
            pred_tmp =  torch.argmax(outputs.data, dim=1) #torch.max(outputs.data, dim=1)
            pred_total.append(pred_tmp)
            targets_total.append(targets)
            # 1/0
            
    #         n_correct += calcuate_accuracy(big_idx, targets)

    #         nb_tr_steps += 1
    #         nb_tr_examples+=targets.size(0)
            
    #         if _%5000==0:
    #             loss_step = tr_loss/nb_tr_steps
    #             accu_step = (n_correct*100)/nb_tr_examples
    #             print(f"Validation Loss per 100 steps: {loss_step}")
    #             print(f"Validation Accuracy per 100 steps: {accu_step}")
    # epoch_loss = tr_loss/nb_tr_steps
    # epoch_accu = (n_correct*100)/nb_tr_examples
    # print(f"Validation Loss Epoch: {epoch_loss}")
    # print(f"Validation Accuracy Epoch: {epohaccu}")
    #print(f"This is the pred total: {pred_total}")
    #print(f"This is the targets_total: {targets_total}")
    # return torch.reshape(torch.stack(pred_total),(-1,)), torch.reshape(torch.stack(targets_total),(-1,))

    return pred_total, targets_total


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
predictions, targets = my_valid(model, testing_loader)

pred_list =[]
for t in predictions:
  for v in t:
      pred_list.append(v.item())

targets_list =[]
for t in targets:
  for v in t:
      targets_list.append(v.item())

# predictions_numpy = predictions.cpu().detach().numpy()
# targets_numpy = targets.cpu().detach().numpy()
# print(predictions_numpy)
# print(targets_numpy)



In [ ]:
#tmp_pred= [v.item() for v in predictions]
#tmp_targets= [v.item() for v in targets]

In [ ]:
recall_score(pred_list,targets_list)


In [ ]:
precision_score(pred_list,targets_list)


In [ ]:
f1_score(pred_list,targets_listy)


In [ ]:
accuracy_score(pred_list,targets_list)

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model. 

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [ ]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')
print('This tutorial is completed')